## LSTM

In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

In [3]:
import pickle

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [5]:
max_features = 20000
maxlen = 20
batch_size = 32

In [6]:
def load_ds(path):
    vecfile = open(path,'r')
    return pickle.load(vecfile)

In [7]:
ds = load_ds('./data/ann/vectors_50_50.txt')
ds.head()

,category,tweet_sec
0,1,"[364, 791, 175, 1, 507, 4768]"
1,1,"[5831, 9, 2338]"
2,0,"[54, 138, 34, 2, 15, 40, 2874, 9, 6389, 4057, ..."
3,1,"[102, 36, 17, 357, 12, 782, 7, 154, 296, 7, 9,..."
4,1,"[872, 112, 29, 595, 29, 535]"


In [8]:
print('Pad sequences (samples x time)')
tweet_sec = sequence.pad_sequences(ds['tweet_sec'], maxlen=maxlen)
print('tweet shape:', tweet_sec.shape)

Pad sequences (samples x time)
('tweet shape:', (15306, 20))


In [9]:
def build_arch():
    print('Build model...')
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [15]:
def cross_val(tweet_sec, target, nFolds):
    kFold = StratifiedKFold(n_splits=nFolds, shuffle=True)
    scores = []
    for train, test in kFold.split(tweet_sec,target):
        model = build_arch()
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(tweet_sec[train],target[train],batch_size=batch_size,epochs=20,verbose=1)
        score, acc = model.evaluate(tweet_sec[test], target[test],batch_size=batch_size)
        scores.append(acc * 100)
    return scores   

In [17]:
results = cross_val(tweet_sec,ds['category'],10)

Build model...
Epoch 1/20
13774/13774 [==============================] - 30s - loss: 0.3869 - acc: 0.8298    
Epoch 2/20
13774/13774 [==============================] - 30s - loss: 0.1897 - acc: 0.9299    
Epoch 3/20
13774/13774 [==============================] - 30s - loss: 0.1144 - acc: 0.9594    
Epoch 4/20
13774/13774 [==============================] - 30s - loss: 0.0763 - acc: 0.9731    
Epoch 5/20
13774/13774 [==============================] - 30s - loss: 0.0536 - acc: 0.9808    
Epoch 6/20
13774/13774 [==============================] - 30s - loss: 0.0443 - acc: 0.9845    
Epoch 7/20
13774/13774 [==============================] - 30s - loss: 0.0316 - acc: 0.9890    
Epoch 8/20
13774/13774 [==============================] - 30s - loss: 0.0292 - acc: 0.9892    
Epoch 9/20
13774/13774 [==============================] - 30s - loss: 0.0261 - acc: 0.9917    
Epoch 10/20
13774/13774 [==============================] - 30s - loss: 0.0179 - acc: 0.9940    
Epoch 11/20
13774/13774 [=========

13776/13776 [==============================] - 29s - loss: 0.0493 - acc: 0.9820    - ETA: 2s -  - ETA: 0s - loss: 0.0494 - acc: 0.
Epoch 6/20
13776/13776 [==============================] - 29s - loss: 0.0380 - acc: 0.9880    
Epoch 7/20
13776/13776 [==============================] - 29s - loss: 0.0282 - acc: 0.9906    
Epoch 8/20
13776/13776 [==============================] - 29s - loss: 0.0215 - acc: 0.9940    - ETA: 1s - loss: 0.0218 - acc: 0.9 - ETA: 1s - loss: 0.
Epoch 9/20
13776/13776 [==============================] - 29s - loss: 0.0225 - acc: 0.9917    
Epoch 10/20
13776/13776 [==============================] - 30s - loss: 0.0135 - acc: 0.9964    
Epoch 11/20
13776/13776 [==============================] - 30s - loss: 0.0203 - acc: 0.9935    
Epoch 12/20
13776/13776 [==============================] - 30s - loss: 0.0130 - acc: 0.9959    
Epoch 13/20
13776/13776 [==============================] - 31s - loss: 0.0129 - acc: 0.9949    
Epoch 14/20
13776/13776 [========================

In [27]:
print("Acc: %.2f (+/- %.2f)" %(np.mean(results), np.std(results)))

Acc: 86.19 (+/- 0.91)


In [34]:
for i in range (0,len(results)):
    print "Model %d, acc: %.2f " %(i+1,results[i])

Model 1, acc: 86.68 
Model 2, acc: 85.64 
Model 3, acc: 87.21 
Model 4, acc: 84.84 
Model 5, acc: 86.54 
Model 6, acc: 84.44 
Model 7, acc: 87.32 
Model 8, acc: 86.54 
Model 9, acc: 86.73 
Model 10, acc: 86.01 
